# Step 2. Collect OSM data via Overpass

In [ ]:
import os
import datetime

import numpy as np
import scipy
import fiona
import statistics
import math

import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, shape, mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union

import matplotlib.pyplot as plt
from matplotlib import colors, cm, style
import matplotlib.patches as mpatches
# from descartes import PolygonPatch

import osmnx as ox
import networkx as nx

import rasterio
from rasterio import MemoryFile
from rasterio.plot import show
from rasterio.mask import mask
import json

import contextily as cx
import folium
from folium.features import DivIcon

In [ ]:
from my_osm import get_osm_data

## Define city and other settings

#### Place |Country    |CRS
Rotterdam    (The Netherlands)    EPSG:28992

Barcelona     (Spain)              EPSG:25830

Goteborg        (Sweden)             EPSG:3006

In [ ]:
place_name = 'Rotterdam'
local_crs = 'EPSG:28992'

In [ ]:
osm_crs = 'EPSG:3857'

In [ ]:
min_area = 200

In [ ]:
export_folder = 'data'

In [ ]:
buffer = 500

## Load OSM data

In [ ]:
place = ox.geocode_to_gdf(place_name).to_crs(local_crs)

extent = place.buffer(buffer, join_style=2)
if len(extent) > 1:
    extent_shape = MultiPolygon([shape(part) for part in extent.to_crs('EPSG:4326')])     
else:
    extent_shape = extent.to_crs('EPSG:4326')[0]

### Classified as greenspace

In [ ]:
park_query = '["leisure"~"^(park)"]'
parks = get_osm_data(park_query, extent, 'convexhull')

In [ ]:
green_leisure_query = '["leisure"~"^(park|nature_reserve)"]'
greenspaces = get_osm_data(green_leisure_query, extent, 'convexhull')

In [ ]:
# excluding: allotments, flowerbed, farmland, farmyard, orchards, vineyard, plant_nursary (i.e. crop production, not for entering)
green_landuse_query = '["landuse"~"^(meadow|grass|village_green|forest)"]'
greenspaces = pd.concat([greenspaces, get_osm_data(green_landuse_query, extent, 'convexhull')])

In [ ]:
green_natural_query = '["natural"~"^(wood|scrub|heath|grassland|fell|shrubbery)"]'
greenspaces = pd.concat([greenspaces, get_osm_data(green_natural_query, extent, 'convexhull')])

### Classified as open space

#### Squares, including pedestrian areas and marketplaces

In [ ]:
# squares (square, marketplace, pedestrian way)
square_place_query = '["place"~"^square"]'
squares = get_osm_data(square_place_query, extent, 'convexhull')

In [ ]:
square_highway_query = '["highway"~"^pedestrian"]["area"~"^yes"]'
squares = pd.concat([squares, get_osm_data(square_highway_query, extent, 'convexhull')])

In [ ]:
square_amenity_query = '["amenity"~"^(marketplace)"]'
squares = pd.concat([squares, get_osm_data(square_amenity_query, extent, 'convexhull')])

In [ ]:
square_leisure_query = '["leisure"~"^(common)"]'
squares = pd.concat([squares, get_osm_data(square_leisure_query, extent, 'convexhull')])

#### Playgrounds, non-private schoolyards, recreation grounds

In [ ]:
# playgrounds/recreation grounds
play_query = (f'["leisure"~"playground"]["access"!="private"]')
playspaces = get_osm_data(play_query, extent, 'convexhull')

In [ ]:
schoolyards_filter = (f'["leisure"~"schoolyard"]["access"!="private"]')
schoolyards = get_osm_data(schoolyards_filter, extent, 'convexhull')

#### Streets

In [ ]:
# custom filter: using the osmnx network_type 'walk' as a basis,
# but NOT excluding 'cycleways'.
# and excluding segments in tunnels
# see https://github.com/gboeing/osmnx/blob/main/osmnx/downloader.py
# and https://github.com/gboeing/osmnx/issues/169
network_filter = (
        f'["highway"]["area"!~"yes"]["access"!~"private"]'
        f'["highway"!~"abandoned|bus_guideway|construction|motor|planned|platform|proposed|raceway"]'
        f'["foot"!~"no"]["service"!~"private"]'
        f'["tunnel"!="yes"]'
    )

In [ ]:
G = ox.graph_from_polygon(extent_shape, network_type='walk', custom_filter=network_filter, retain_all=True)
G = ox.project_graph(G, to_crs=local_crs)

In [ ]:
streets = ox.utils_graph.graph_to_gdfs(ox.get_undirected(G), nodes=False, edges=True).to_crs(local_crs)

### Classified as non-greenspace

In [ ]:
# parkings
parking_query = '["amenity"~"^(parking)"]["tunnel"!="yes"]["parking"!="underground"]'
parkings = get_osm_data(parking_query, extent, 'convexhull')

### Visualize all

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12,12))

greenspaces.plot(ax=axs, color='green')

squares.plot(ax=axs, color='pink')
playspaces.plot(ax=axs, color='orange') 
streets.plot(ax=axs, color='purple', linewidth=0.2)

parkings.plot(ax=axs, color='black')

place.plot(ax=axs, facecolor='none', edgecolor='#dd1c77', linestyle='-', linewidth=5, zorder=4)
cx.add_basemap(ax=axs, crs=local_crs, source=cx.providers.OpenStreetMap.Mapnik, alpha=0.5, zorder=0)

bbox = place.total_bounds
margin = 1000
xlim = ([bbox[0]-margin, bbox[2]+margin])
ylim = ([bbox[1]-margin, bbox[3]+margin])
axs.set_xlim(xlim)
axs.set_ylim(ylim)
plt.axis('off')

plt.show()

### Filter collected spaces on area and public access

In [ ]:
def space_filter(gdf, local_crs, union=False, union_threshold=0, place=None, place_threshold=0, place_predicate='intersects', min_area=10000, G=None, G_threshold=0):
    """
    gdf = geoseries of greenery polygons (e.g. greenery.geometry)
    return = a union of filtered polygons
    [optional] threshold to merge spaces in proximity (in meters)
    [optional] place geoseries of total area geometries (e.g. place_gdf.geometry)
    [optional] minimum area for greenspace filter, default value set
    [optional] graph which greenspace should intersect nodes and edges of
    """
    
    filtered = gdf.copy()

    if 'tags' in filtered.columns:
        filtered['tags'] = filtered['tags'].astype(str)
    
    # only polygons, no points or lines
    filtered = filtered[(filtered.geom_type == 'Polygon')|(filtered.geom_type == 'MultiPolygon')]
    
    #union of adjacent and overlapping polygons
    if union:
        filtered['geometry'] = filtered.buffer(union_threshold)
        filtered = gpd.GeoDataFrame(filtered.unary_union).rename(columns={0:'geometry'}).set_geometry('geometry').set_crs(local_crs)
        filtered['geometry'] = filtered.buffer(-union_threshold)

    # filter on intersect with place
    if place is not None:
        place_temp = place.copy().set_geometry(place.geometry.buffer(place_threshold))
        filtered = gpd.sjoin(filtered, place_temp, how="left", predicate=place_predicate)
        filtered = filtered[filtered.index_right.notnull()].drop(columns=['index_right'])
        
    # filter on minimum area
    if min_area > 0:
        filtered = filtered[filtered.geometry.area>=min_area]
    
    # filter on intersection with network
    if G:
        edges = ox.utils_graph.graph_to_gdfs(G, nodes=False, edges=True)
        if G_threshold:
            edges = edges.copy().set_geometry(edges.geometry.buffer(G_threshold))
        filtered = gpd.sjoin(filtered, gpd.GeoDataFrame(edges.reset_index(drop=True).geometry), how="left", predicate='intersects')
        filtered = filtered[filtered.index_right.notnull()].drop(columns=['index_right'])
        
        filtered.drop_duplicates(inplace=True)
        
        nodes = ox.utils_graph.graph_to_gdfs(G, nodes=True, edges=False)
        if G_threshold:
            nodes = nodes.copy().set_geometry(nodes.geometry.buffer(G_threshold))
        filtered = gpd.sjoin(filtered, gpd.GeoDataFrame(nodes.reset_index(drop=True).geometry), how="left", predicate='intersects')
        filtered = filtered[filtered.index_right.notnull()].drop(columns=['index_right'])

    filtered.drop_duplicates(inplace=True)
    return filtered

In [ ]:
# all places larger than 0.02 sq.m. and intersecting with the pedestrian network
parks_filtered = space_filter(parks, local_crs, place=place, place_threshold=buffer, min_area=min_area, G=G)
greenspaces_filtered = space_filter(greenspaces, local_crs, place=place, place_threshold=buffer, min_area=min_area, G=G)

# for squares and play spaces, allow intersection with 10m threshold, as often not crossed but just directly adjacent
# but still meant for humans to access (contrary to some greenspaces, e.g., grass or bushes in between streets) 
squares_filtered = space_filter(squares, local_crs, place=place, place_threshold=buffer, min_area=min_area, G=G, G_threshold=10)
playspaces_filtered = space_filter(playspaces, local_crs, place=place, place_threshold=buffer, min_area=min_area, G=G, G_threshold=10)
# streets not filtered, as they are publicly accessible by definition and do not have an area

parkings_filtered = space_filter(parkings, local_crs, place=place, place_threshold=buffer, min_area=min_area, G=G)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12,12))

greenspaces_filtered.plot(ax=axs, color='green')

squares_filtered.plot(ax=axs, color='pink')
playspaces_filtered.plot(ax=axs, color='orange') 
streets.plot(ax=axs, color='purple', linewidth=0.2)

parkings_filtered.plot(ax=axs, color='black')

place.plot(ax=axs, facecolor='none', edgecolor='#dd1c77', linestyle='-', linewidth=5, zorder=4)
cx.add_basemap(ax=axs, crs=local_crs, source=cx.providers.OpenStreetMap.Mapnik, alpha=0.5, zorder=0)

bbox = place.total_bounds
margin = 500
xlim = ([bbox[0]-margin, bbox[2]+margin])
ylim = ([bbox[1]-margin, bbox[3]+margin])
axs.set_xlim(xlim)
axs.set_ylim(ylim)
plt.axis('off')

plt.show()

In [ ]:
print("""
In {}:

- {} greenspaces ({} parks)

- {} squares
- {} play and recreation grounds
- {} street segments

- {} parkings

That are larger than {} sq.m. and publicly accessible by foot.
""".format(
    place_name,
    len(greenspaces_filtered),
    len(parks_filtered),
    len(squares_filtered),
    len(playspaces_filtered),
    len(streets),
    len(parkings_filtered),
    min_area
))

## Export OSM data

In [ ]:
place_name_out = place_name.split(',')[0].replace(' ', '')
today = datetime.date.today().strftime("%d%b%Y")
export_sub_folder = os.path.join(export_folder, 'OSM', '{}_{}'.format(place_name_out, today))

In [ ]:
if not os.path.exists(export_sub_folder):
    os.mkdir(export_sub_folder)

In [ ]:
output_file = os.path.join(export_sub_folder, 'parks.geojson')
parks_export = parks_filtered.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
parks_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'greenspaces.geojson')
greenspaces_export = greenspaces_filtered.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
greenspaces_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'squares.geojson')
squares_export = squares_filtered.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
squares_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'playspaces.geojson')
playspaces_export = playspaces_filtered.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
playspaces_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'streets.geojson')
streets_export = streets.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
streets_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'parkings.geojson')
parkings_export = parkings_filtered.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
parkings_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'place.geojson')
place_export = place.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
place_export.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = os.path.join(export_sub_folder, 'G.graphml')
ox.save_graphml(G, output_file)